# 1. Data Exploration

### Information on the Dataset

Data Fields for SNOW T15 and SNOW T23 ⛄<br>
Resource: https://huggingface.co/datasets/snow_simplified_japanese_corpus <br>
Paper: https://aclanthology.org/L18-1072.pdf

- <strong>ID</strong>: sentence ID.
- <strong>original_ja</strong>: original Japanese sentebolnce.
- <strong>simplified_ja</strong>: simplified Japanese sentence.
- <strong>original_en</strong>: original English sentence.
- <strong>proper_noun</strong>: (included ONLY in SNOW T23) Proper nowus that the workers has extracted as proper nouns. The authors instructed workers not to rewrite proper nouns, leaving the determination of proper nouns to the workers.

# 2. Baseline Model

In the SNOW T15 dataset it states: <br>
<i>Core vocabulary is restricted to 2,000 words where it is selected by accounting for several factors such as meaning preservation, variation, simplicity and the UniDic word segmentation criterion/</i>

#### Step 1: Take a sample size from the SNOW T15 dataset and extracted 2,000 simplified terms.

In [ ]:
# """ Required installations """

# !pip install mecab-python3
# #These wheels include a copy of the MeCab library, but not a dictionary. 
# #In order to use MeCab you'll need to install a dictionary. unidic-lite is a good one to start with:
# !pip install unidic-lite

# # normalization tool
# !pip install neologdn

# !pip install openpyxl

# # To be able to see in Japanese!
# !pip install japanize_matplotlib

In [44]:
import os
import pandas as pd
import numpy as np
import re

# Preprocessing
import MeCab
import neologdn
import collections
from nltk import FreqDist
from nltk.corpus import stopwords

import time
import logging
import collections
import logging
import time
from gensim.models.word2vec import Word2Vec
logging.basicConfig()
logging.root.setLevel(logging.INFO)

# Visualization
import matplotlib.pyplot as plt
import japanize_matplotlib

In [2]:
def get_data(file):
    """
    Gets csv data under 'simply-japanese/data/'
    Returns as Dataframe where columns=['original','simplified']
    """

    # FIXME:  Make sure to
    # 1. Change these when you transfer to .py file
    # 2. Put these global variables somewhere else
    
    CURRENT_PATH = 'notebooks/baseline_workflow.ipynb'
    DATA_PATH = 'data/2_RawData'
    csv_path = os.path.abspath(__file__)[:-len(CURRENT_PATH)]  + DATA_PATH
    df = pd.read_excel(os.path.join(csv_path, file))
    
    df.drop(columns=['#英語(原文)','#固有名詞'], inplace=True, errors='ignore')
    df.rename(columns={"#日本語(原文)": "original", "#やさしい日本語": "simplified"}, inplace=True)
    
    return df

In [3]:
# FIXME: Set df in __init__ 
def term_frequency(df, col='original'):
    """
    Count number of terms in a corpus
    Ignore independent words  ["助動詞", "助詞", "補助記号"] and words in japanese stopwords
    Returns collection of term and its frequency
    """
    # FIXME : Need to find a way to implement japanese_stopword.txt when this file is used externally
    jp_stopwords = stopwords.words('japanese')
    all_terms = collections.Counter()
    t = MeCab.Tagger("-O wakati")
    for idx, row in df.iterrows():
        text = row[col]
        node = t.parseToNode(text).next
        while node.next:
            part_of_speech = node.feature.split(',')[0]
            # TBD
            if part_of_speech in ["助動詞", "助詞", "補助記号"] or node.surface in jp_stopwords:
                node = node.next
                continue
            all_terms[node.surface] += 1
            node = node.next
    return all_terms

In [4]:
def get_simplified_terms(df, n_most_common):
    """
    Only returns simplified terms that exists in the simplified column
    Return list until the top 'n' elements from most common
    """
    # Filter out corpuses if original and simplified are exactly the same
    diff_corpus_df = df[df['original'] != df['simplified']]
    
    # Create collections of original and simplified terms
    original_terms = term_frequency(diff_corpus_df, 'original')
    simplified_terms = term_frequency(diff_corpus_df, 'simplified')
    
    # Compare two collections using subtract
    diff_terms = simplified_terms
    diff_terms.subtract(original_terms)
    
    diff_terms_df = pd.DataFrame(dict(diff_terms).items(), columns=['word', 'count'])
    return diff_terms_df[diff_terms_df['count'] >= 0].sort_values(by='count', ascending=False)['word'].tolist()[:n_most_common]

#### Step 2: Using the 2000 list of simplified terms from Step 1, find the nearest term

In [ ]:
"""
tf_list = 2000 simplified term frequency retrieved from data
pos_list = specified list of POS (Parts-Of-Speech)

1. Go through each row in the original data
2. Check it word is in the pos_list
3. Check if a word is in the tf_list
    if yes, continue to the next word
4. If 2. is no: check the similarity of the word with all the tf_list
5. Replace the word with maximum value and if the maximium exceeds a specified threshold

"""

"""
PSEUDO CODE

threshold = minimum similarity
for sentence in data:
    for word in sentence:
        if word.pos in pos_list:
            if word in tf_list:
                continue
            else:
                for tf in tf_list:
                    list = []
                    list.append(wv.similarity(word, tf))
                replace word with max(list) if max(list) > threshold
        else: continue
"""

In [47]:
# new version that can handle romaji, also changed the way it handles numerals

def is_romaji(string):
    """
    returns True if string contains Romaji or a number, False if not.
    """
    alphanum_full = r'[A-z0-9]'
    if re.findall(alphanum_full, string):
        return True
    return False

def replace_terms(data, term_list, wv):
    """
    1. Identify every POS in a sentence and if it should be replaced
    2. Use the pre-trained Word2Vec model to get a term from term_list with closest distance to POS
    3. Replace POS in sentence
    4. Add new sentence to dataframe in column "prediction"

    input:
    data, np.series
    term_list, list of simplified terms
    wv, word2vec model.wv

    output:
    prediction, np.series
    """
    logging.root.setLevel(logging.INFO)

    start = time.time()
    # Make sure the data is a series, not a df or list
    try:
        assert type(data) == pd.core.series.Series
        logging.info("Data file type OK")
    except:
        print("Data file type is NOT a pd.series")

    pos_list = ("名詞", "動詞", "代名詞") # POS (part of speech) that will possibly be removed
    threshold = 0.5 # Threshold of similarity, over which a term will be replaced
    t = MeCab.Tagger()
    counter = collections.Counter()
    prediction = data.copy()
    assert len(prediction) == len(data)     # Make sure prediction and data have the same size

    # Iterate over every sentence in the dataset
    for idx, row in data.items():
        row = neologdn.normalize(row)
        logging.debug(f"Currrent sentence: {row}")
        sentence = []
        # Iterate over every word in the sentence
        node = t.parseToNode(row).next
        while node.next:
            if is_romaji(node.surface):
                sentence.append(node.surface)
                node = node.next
            word = node.feature.split(',')[8]
            part_of_speech = node.feature.split(',')[0]
            # If POS is not noun, pronoun or verb: add word to list and continue
            if part_of_speech not in pos_list:
                sentence.append(word)
            else:
                # If the term is already in the term list: do not replace, add word to list and continue
                if word in term_list:
                    sentence.append(word)
                else:
                    # Replace word with closest word from term list
                    try:
                        if wv.most_similar(word)[0][1] > threshold:
                            closest_word = wv.most_similar(word)[0][0]
                            sentence.append(closest_word)
                        else:
                            sentence.append(word)
                    except KeyError as e:
                        sentence.append(word)
                        logging.warning(f"{e}. Term will not be replaced.")
            counter[node.surface] += 1
            node = node.next
        logging.debug(sentence)
        prediction[idx] = "".join(sentence)

    assert len(data) == len(prediction)  # Make sure prediction and data have the same size
    end = time.time()
    logging.info(end-start)
    return prediction

In [40]:
X150 = get_data('SNOW_T15_150.xlsx')

In [41]:
X150.head()

,original,simplified
0,月曜日までにこの仕事を終えて下さい。,月曜日までにこの仕事を終わらせてください。
1,失敗してもあきらめてはいけない。,失敗してもダメと思ってはならない。
2,あなたは何を見つめているのですか。,あなたは何を見ているのですか。
3,その女の子は母と似ていた。,その少女は母と似ていた。
4,彼は貧しかったので、大学へ行けなかった。,彼はお金がなかったので、大学へ行くことができなかった。


In [49]:
term_list = get_simplified_terms(X150, 2000)
model = Word2Vec.load("word2vec.gensim.model")

INFO:gensim.utils:loading Word2Vec object from word2vec.gensim.model
INFO:gensim.utils:loading wv recursively from word2vec.gensim.model.wv.* with mmap=None
INFO:gensim.utils:loading syn0 from word2vec.gensim.model.wv.syn0.npy with mmap=None
INFO:gensim.utils:loading syn1neg from word2vec.gensim.model.syn1neg.npy with mmap=None
INFO:gensim.utils:setting ignored attribute syn0norm to None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:Word2Vec lifecycle event {'fname': 'word2vec.gensim.model', 'datetime': '2023-02-18T17:47:59.747479', 'gensim': '4.3.0', 'python': '3.8.12 (default, Sep 24 2022, 11:39:20) \n[GCC 9.4.0]', 'platform': 'Linux-5.10.16.3-microsoft-standard-WSL2-x86_64-with-glibc2.29', 'event': 'loaded'}


In [50]:
predictions = replace_terms(X150['original'], term_list=term_list, wv=model.wv)

INFO:root:Data file type OK
INFO:root:9.060764789581299


In [51]:
predictions[:10]

0     月曜時刻までにこの仕事を終えるてください。
1           失敗図っても諦めては構わない。
2       あなたは幾を思い浮かべておりのですか。
3             その男の子は母と似てきた。
4    彼自身は貧しかったので、大学へ歩けなかった。
5           彼女自身はとても運転がざらだ。
6           どんな寝室でもないよりはよい。
7      クラスメートがそんな事を言った筈がない。
8        あたしが成功図ったと聞いてうれしい。
9            貴方の父はクラスメートです。
Name: original, dtype: object

In [54]:
d = {'predict': predictions, 'simplified': X150['simplified'].to_list()}
pred_df = pd.DataFrame(data=d)
pred_df.tail()

,predict,simplified
145,彼女自身は諺をいくつも知っており。,彼女は昔から知られてい短い言葉をいくつも知っている。
146,貴方たちのチームが勝つ事を断言図ってきます。,私たちのチームが勝つと思っている。
147,貴方は日本の前史に関心があり。,私は日本の歴史に興味がある。
148,貴方は時折学校で彼女自身に会う。,私は時々学校で彼女に会う。
149,客車は何処で乗らますか。,列車はどこで乗ることができますか。


In [53]:
X150['original']

0         月曜日までにこの仕事を終えて下さい。
1           失敗してもあきらめてはいけない。
2          あなたは何を見つめているのですか。
3              その女の子は母と似ていた。
4       彼は貧しかったので、大学へ行けなかった。
               ...          
145       彼女はことわざをいくつも知っている。
146    私たちのチームが勝つことを確信しています。
147           私は日本の歴史に興味がある。
148            私は時折学校で彼女に会う。
149             電車はどこで乗れますか。
Name: original, Length: 150, dtype: object

# 3. Evaluation Metric

### 3.1) WER SCORE

In [34]:
def wer_score(predicted, simplified, debug=True):
    '''
    Compares the simplified ML prediction of a given text to the pre-existing simplified
    text given with the dataframe.
    Using the WER (word error rate) algorithm.
    Adds the WER score as a new column to the Dataframe
    '''
    r = predicted.split()
    h = simplified.split()
    costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3
    DEL_PENALTY = 1
    INS_PENALTY = 1
    SUB_PENALTY = 1
    for i in range(1, len(r)+1):
        costs[i][0] = DEL_PENALTY*i
        backtrace[i][0] = OP_DEL
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS
    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
                insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
                deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1
                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL
    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    if debug:
        lines = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i-=1
            j-=1
            if debug:
                lines.append("OK\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub +=1
            i-=1
            j-=1
            if debug:
                lines.append("SUB\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j-=1
            if debug:
                lines.append("INS\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i-=1
            if debug:
                lines.append("DEL\t" + r[i]+"\t"+"****")
    return (numSub + numDel + numIns) / (float) (len(r))
    wer_result = round( (numSub + numDel + numIns) / (float) (len(r)), 3)
    
    
def wer_jp(original, simplified):
    ori = ''
    simpi = ''
    for i in original:
        ori += i + ' '
    for i in simplified:
        simpi += i + ' '
    return round(wer_score(ori, simpi),3)


def evaluate_wer_score(df):
    wer_list = []
    for i in df.index:
        original_text = df.iloc[i][0]
        simplified_text = df.iloc[i][1]
        wer_list.append(wer_jp(original_text, simplified_text))
    df['WER_score'] = wer_list

In [55]:
evaluate_wer_score(pred_df)
pred_df.head(40)

,predict,simplified,WER_score
0,月曜時刻までにこの仕事を終えるてください。,月曜日までにこの仕事を終わらせてください。,0.238
1,失敗図っても諦めては構わない。,失敗してもダメと思ってはならない。,0.600
2,あなたは幾を思い浮かべておりのですか。,あなたは何を見ているのですか。,0.421
3,その男の子は母と似てきた。,その少女は母と似ていた。,0.308
4,彼自身は貧しかったので、大学へ歩けなかった。,彼はお金がなかったので、大学へ行くことができなかった。,0.545
5,彼女自身はとても運転がざらだ。,彼女はとても運転がうまくない。,0.467
6,どんな寝室でもないよりはよい。,どんなベッドでもないよりはよい。,0.200
7,クラスメートがそんな事を言った筈がない。,先生がそんなことを言ったはずがない。,0.500
8,あたしが成功図ったと聞いてうれしい。,あなたが成功したと聞いて嬉しい。,0.333
9,貴方の父はクラスメートです。,私の父は先生です。,0.571


In [56]:
pred_df['WER_score'].mean()

0.4377999999999999

In [57]:
evaluate_wer_score(X150)
X150['WER_score'].mean()

0.2200533333333333

In [ ]:
DEBUG:root:Currrent sentence: 彼は部屋をきちんと整理した。
DEBUG:root:['彼自身', 'は', '部屋', 'を', 'きちんと', '整理', '図っ', 'た', '。']

# 3. Preprocessing

# 3.1) Data Organization and Clean Up!

In [ ]:
# All the imported libraries go here for Section 2